Define standard Houdini parameters and load some extra very common libraries... Attention, under Windows Houdini comes with an older version of Python2.7 and numpy, causing for example np.stack to fail.

In [ ]:
# Fluffy Houdini Node
from __future__ import division
import numpy as np
import math

node = hou.pwd()
geo = node.geometry()
# Add code to modify contents of geo.
# Use drop down menu to select examples.

Next, read all the variables from the parameter interface with 'node.parm(...).eval()' and define some generic ones in the script

In [ ]:
global epsilon, beta, rmin,rmax,relativeR,relativepulse,relativetorsion,looseness,pi, nStrands,nHairs,loopiness,th0Default,maxnumberofloosewindings,fluffyness,reducePoints, jitter, mushheight,mushwidth

reducePoints=node.parm('reducePoints').eval()
rmin=node.parm('radiusx').eval()
rmax=node.parm('radiusy').eval()

relativepulse=1/node.parm('golven').eval()
relativetorsion=node.parm('windingen').eval()

nStrands=node.parm('haartjesx').eval()
nHairs=node.parm('haartjesy').eval()

fluffyness=node.inputs()[1].geometry().attribValue('fluffyness')
#fluffyness=node.parm('fluffyness').eval()
loopiness=node.parm('loopiness').eval()
maxnumberofloosewindings=node.parm('maxloose').eval()
looseness=node.parm('looplooseness').eval()

flip=node.parm('flip').eval()

jitter=node.parm('jitter').eval()

mushheight=math.floor(node.parm('mushroomx').eval())
mushwidth=node.parm('mushroomy').eval()

epsilon=1
beta=0.3

pi=3.1415926
th0Default=np.roll(np.arange(nStrands),3)/nStrands*2*pi

We'll need some fairly standard list/geometry operations

In [ ]:
def cycle(ar,shift):
 return ar.transpose(np.roll(np.arange(len(ar.shape)),shift))
def stack(x,y,z):
 #for lower versions of numpy
 #return np.stack((x,y,z),axis=1)
 return np.transpose(np.array([x,y,z]),(1,0,2))
def polar2vec(r,th,z):
 xyz=np.array([r*np.cos(th),r*np.sin(th),z])
 return cycle(xyz,-1)

We want to 'modulate' the thickness of the yarn slightly waving along its length. Given an initial radius 'r' of a single hair, we modify it slightly in a waving fashion to 'R'. An extra scale parameter will enable us to make loose loops in the hairs.

In [ ]:
def r2R(zs,rs,th0=th0Default,scale=1):
 global rmin, rmax, s, relativetorsion,pi,th0Default
 tau=relativetorsion
 wl=relativepulse
 zunit=zs[-1]
 zA,rA=np.meshgrid(zs,rs)
 return rmin*rA+(rmax-rmin)/2*rA*(1+np.cos(th0+wl*2*pi*zA/zunit))*scale, zA

We define a good distribution of the hairs following the Cornell paper:

In [ ]:
def pdf(R):
 # exponential map [0,1] -> [1,0]
 pp=lambda r: (np.exp(1)-np.exp(r))/(np.exp(1)-1)
 # map [0,1]->[1-eps,eps]
 return (1-2*epsilon)*pp(R)**beta+epsilon

Now comes the heavy lifting by the function 'strands': We define a set of M hairs of which a number (depending on loosness chance) will have increased radius, depicting loosened hairs. 
- First we distribute hairs equally over a circle, but add some random deviation of this regular pattern
- Next we distribute the distance of each hair to the center according to the defined pdf-distribution
- Then we randomly loosen each single hair, with the chance of a certain number of loose windings binomially depending on maxnumberofloosewindings and loopiness, the position of the loose loop is uniformly random.
- Next we modulate the yarn width with the previously defined function and introduce the torsion
- lastly we change the head of the hair: we turn off the torsion and make the hairs spread parabolically
- We output a list of hairs, which in turn are a list of Cartesian coordinates we convert from the polar ones with the previously defined function.

In [ ]:
def strands(zs,M,th0=th0Default):
 global th0Default, maxnumberofloosewindings,looseness,relativetorsion
 
 # define some custom variables
 tau=relativetorsion
 wl=relativepulse
 zunit=zs[-1]
 iis=np.arange(M)

 #add some jitter on the helicial pattern of the hairs
 ths=iis/M*2*pi
 ths+=(0.5-np.random.ranf((len(ths),)))/M*2*pi*jitter
    
 #set of randomly distributed rvalues, according to pdf
 rOptions=np.linspace(0,1,1000)
 rProbs=pdf(rOptions)/sum(pdf(rOptions))
 rStrings=np.random.choice(rOptions,M,p=rProbs)*zunit
    
 #pick loose windings
 windingnrs=np.arange(np.floor(tau).astype('uint32'))
 minimalZindices=np.searchsorted(zs,zunit*np.array([wl*(th0/2/pi+k) for k in windingnrs]))
 firstloosewinding=np.random.choice(windingnrs,M)
 nrofloosewindings=np.random.binomial(maxnumberofloosewindings,loopiness,M)
 looseStart=[np.searchsorted(zs,zunit*wl*(th0[k]/2/pi+firstloosewinding[k])) for k in iis]
 looseEnd=[np.searchsorted(zs,zunit*wl*(th0[k]/2/pi+np.minimum(firstloosewinding[k]+nrofloosewindings[k],windingnrs[-1]-1))) for k in iis]
 loosen=np.ones((M,len(zs)))
 for i in iis:
  loosen[i,looseStart[i]:looseEnd[i]]=looseness

 # define polar coordinates of each point of every hair in the yarn
 R,Z=r2R(zs,rStrings,1,loosen)
 ths2=np.expand_dims(ths,axis=-1)+2*pi*tau*Z/zunit
    
 # correct the uppermost points to open/desentangle the hairs there, like a muchroom
 R[:,-mushheight:]*=1+ (np.arange(mushheight)**2/mushheight**2)*(mushwidth-1)
 ths2[:,-mushheight:]=np.expand_dims(ths2[:,-mushheight],axis=-1)

 return polar2vec(R,ths2,Z)

Now we alredy have a straight yarn with all equal length hairs, we want to add some shorter ones sticking out. The resulting yarn will contain contributions from both 'strands' and 'hairs'. 
- We let the hair exist between 1.5 and 2 times a standard winding, giiven by the original winding times relativetorsion
- We distribute the max radius of each sticking hair between fluffyness x rmax and 2 x fluffyness x rmax, and the begin- and end-angle uniformly
- We output again a list of points in Cartesian coordinates along the full yarn, and a 'chk' list to indicate wich parts are relevant. (For the rest we get unphysical/unimportant contributions that we'll have to ignore later on).

In [ ]:
def hairs(zs,M):
 zunit=zs[-1]
 skip=len(zs)/relativetorsion
 iis=np.arange(M)

 hairStart=np.random.choice(np.arange(len(zs)-1),M)
 hairEnd=[[zs[np.minimum(np.random.choice(np.arange(hairStart[i]+3*skip//2,hairStart[i]+2*skip)),len(zs)-1)]] for i in np.arange(len(iis))]
 hairStart=np.array([zs[k] for k in hairStart])
 zA,hS=np.meshgrid(zs,hairStart)

 rangeR=np.expand_dims(np.array([rmin*np.zeros((M,)),(np.random.rand(M,)+np.ones((M,)))*fluffyness*(rmax-0*rmin)])*zunit,axis=-1)
 rangeTh=np.expand_dims(np.random.rand(2,len(iis))*2*pi,axis=-1)

 f=np.array((zA-hS)/(hairEnd-hS))
 chk=(1+np.sign(f*(1-f)))/2
 return polar2vec((f*rangeR[1]+rangeR[0]),f*rangeTh[1]+rangeTh[0],zA), chk

All yarn contributions are now relative to a straight yarn core. We need to account for bending. Given the coordinates of the bended core, we first define the relative coordinates in a frame rotated according to the bending (technically: Frenet-Serret  vectors 't', 'n' and 'b'.).
- We put the x- and y-coordinate, perpendicular to the straight yarn we defined before, along the normal directions 'n' and 'b', orthogonal to the curved yarn.
- We output the old coordinates shifted by these two contributions.

In [ ]:
def warp(oldvecs,newzs):
 tz=newzs[1:]-newzs[:-1]
 tz=np.concatenate((np.ones((1,3)),tz),axis=0)
 
 gz=tz[1:]-tz[:-1]
 gz=np.concatenate((np.ones((1,3)),gz),axis=0)
 nrm=np.expand_dims(np.linalg.norm(tz,axis=-1),axis=-1)
 nz=gz/nrm-tz*np.expand_dims(np.sum(gz*tz,axis=-1),axis=-1)/(nrm**3)
 nz[0]=np.ones((3,))
 tz=tz/nrm
 nrm=np.expand_dims(np.linalg.norm(nz,axis=-1),axis=-1)
 nz=nz/nrm
 nz[0]*=0

 bz=np.cross(tz,nz)
    
 fs=np.expand_dims(stack(nz,bz,0*tz),axis=0)
 ov=np.expand_dims(oldvecs,axis=-1)
 return np.sum(ov*fs,axis=2)+newzs

We want to make from a given (curved) list of points, a set of polylines (by which we mean a list of Cartesian points) forming the yarn we want. 
- First we defile the lengt of an equivalent straight yarn 'lPoly'
- We warp all long strands around it by using subsequently the previously defined 'strands' (creating random sdistributed hairs) and 'warp' (curving them).
- We do the same for the shorter hairs sticking out with 'hairs' and 'warp'. Remember that a number of contributions are unphysical, given by the accompanying check-list.
- We combine all hairs and a corresponding global check-list, which selects all strands-points but only part of the hairs-points.

In [ ]:
def poly2polys(poly):
 global nStrands, nHairs, mushheight
 lPoly=np.sum(np.linalg.norm(poly[1:]-poly[:-1]))
    
 zs=np.linspace(0,lPoly,len(poly))
 strandvecs=warp(strands(zs,nStrands),poly)
 chk1=np.ones((strandvecs.shape[0],strandvecs.shape[1]))

 hrs,chk2=hairs(zs,nHairs)
 hairvecs=warp(hrs,poly)

 allvecs=np.concatenate((strandvecs,hairvecs))
 allchks=np.concatenate((chk1,chk2))
 return allvecs,allchks

We create a single strand in Houdini, taking into account a check-list that might eliminate some unphysical points. Also, we oversampled the yarn to nicely capture the curvature in calculation, but we may however want to keep the redundant points out of Houdini by keeping only one in 'reducepoints'.

In [ ]:
def newpoly(pointlist,chks):
 global mushheight
 poly=geo.createPolygon()
 poly.setIsClosed(0)
 i=0
 for (p,c) in zip(pointlist,chks):
  cutoff=np.random.randint(len(pointlist)-mushheight,len(pointlist))
  if (((c>0) and (i%reducePoints==0 or i==len(pointlist)-1)) and i<=cutoff):
   pt=geo.createPoint()
   pt.setPosition(p)
   poly.addVertex(pt)
  i+=1
 return poly

We create a group of predefined polylines in Houdini that constitute our yarn.

In [ ]:
def newyarn(polylist,chklist):
 global mushheight
 yarn=geo.createPrimGroup("yarn")
 for (pointlist,chks) in zip(polylist,chklist):
  poly=newpoly(pointlist,chks)
  yarn.add(poly)
 return yarn

This is the actual workflow:
    - We read in coordinates of given Houdini points
    - We make sure the top is at the end (controlled by hand through flip)
    - We create a set of individual hairs out of it as Houdini polylines, nicely curved around the original one, but with a check-list potentially eliminating some points.
    - We group the hairs in a yarn as a Houdini primitive group.  

In [ ]:
inputPolyline=np.array([[p.position().x(),p.position().y(),p.position().z()] for p in geo.points()])
if flip:
 inputPolyline=inputPolyline[::-1]
polylist,chklist=poly2polys(inputPolyline)
newyarn(polylist,chklist)

# Addendum: generating twisting wires

We want to be able to generate the polylines that are fed to the above script ourselves. 

In [ ]:
global relativetorsion,pi,  mushheight,mushwidth

r0=node.parm('radius').eval()

relativetorsion=node.parm('windingen').eval()

nStrands=node.parm('strands').eval()

mushheight=math.floor(node.parm('mushroomx').eval())
mushwidth=node.parm('mushroomy').eval()
 
pi=3.1415926

# In[3]:


def cycle(ar,shift):
    return ar.transpose(np.roll(np.arange(len(ar.shape)),shift))
 
def polar2vec(r,th,z):
    xyz=np.array([r*np.cos(th),r*np.sin(th),z])
    return cycle(xyz,-1)

def stack(x,y,z):
    #return np.stack((x,y,z),axis=1)
    return np.transpose(np.array([x,y,z]),(1,0,2))

def warp(oldvecs,newzs):
    tz=newzs[1:]-newzs[:-1]
    tz=np.concatenate((np.ones((1,3)),tz),axis=0) 
    nrmt=np.expand_dims(np.linalg.norm(tz,axis=-1),axis=-1)
    
    gz=tz[1:]-tz[:-1]
    gz=np.concatenate((np.ones((1,3)),gz),axis=0)
    nz=gz/nrmt-tz*np.expand_dims(np.sum(gz*tz,axis=-1),axis=-1)/(nrmt**3)
    nz[0]=np.ones((3,))
    nrmn=np.linalg.norm(nz,axis=-1)
    tz /= nrmt
    #regularize
    tresh=0.0001
    b0=np.cross(np.mean(tz,axis=0),np.mean(gz,axis=0))
    b0 /= np.linalg.norm(b0)
    if np.all(np.abs(b0)<10^(-8)):
        b0=np.array([1,0,0])
    if np.any(nrmn<tresh):
        nz[nrmn<tresh]=np.cross(b0,tz[nrmn<tresh],axis=-1)
    nrmn[nrmn<tresh]=1
    ##########
    nz *= np.expand_dims(np.sign(np.cross(tz,nz,axis=-1).dot(b0)) / nrmn,1)
    nz[0]*=0
    bz=np.cross(tz,nz)
    fs=np.expand_dims(stack(nz,bz,0*tz),axis=0)
    ov=np.expand_dims(oldvecs,axis=-1)
    return np.sum(ov*fs,axis=2)+newzs

def poly2polys0(poly,r0,M,tau,mushheight,mushwidth):
    rs =np.ones((M,))*r0
    lPoly=np.sum(np.linalg.norm(poly[1:]-poly[:-1]))
    zs=np.linspace(0,lPoly,len(poly))
 
    # define some custom variables
    zunit=zs[-1]
    ths=np.arange(M)/M*2*pi

    # define polar coordinates of each point of every hair in the yarn
    Z,R=np.meshgrid(zs,rs)
    ths2=np.expand_dims(ths,axis=-1)+2*pi*tau*Z/zunit
    
    # correct the uppermost points to open/desentangle the hairs there, like a muchroom
    R[:,-mushheight:]*=1+ (np.arange(mushheight)**2/mushheight**2)*(mushwidth-1)
    ths2[:,-mushheight:]=np.expand_dims(ths2[:,-mushheight],axis=-1)

    return warp(polar2vec(R,ths2,Z),poly)
    

def newpoly(pointlist,chks):
    global mushheight
    poly=geo.createPolygon()
    poly.setIsClosed(0)
    i=0
    for p in pointlist:
        pt=geo.createPoint()
        pt.setPosition(p)
        poly.addVertex(pt)
        i+=1
    return poly 

def newyarn(polylist,chklist,twisted):
    global mushheight
    yarn=geo.createPrimGroup("yarn") 
    first=True
    for (pointlist,chks,i) in zip(polylist,chklist,range(nStrands)):   
        if first:
            first = False
            poly=geo.createPolygon()
            poly.setIsClosed(0)
            poly0=geo.points()
            for (p,pos,j) in zip(poly0,pointlist,range(len(poly0))):
                p.setPosition(pos)
                poly.addVertex(p) 
        else:
            poly=newpoly(pointlist,chks)
        twisted[i].add(poly)
        yarn.add(poly)
    return yarn  
   
inputPolyline=np.array([[p.position().x(),p.position().y(),p.position().z()] for p in geo.points()])
twisted=[]
for i in range(nStrands):
    twisted += [geo.createPrimGroup("twisted"+str(i+1))]
polylist=poly2polys0(inputPolyline,r0,nStrands,relativetorsion,mushheight,mushwidth)
chklist = np.ones(polylist.shape[:2])
newyarn(polylist,chklist,twisted)
